In [ ]:
%%html
<marquee style='width: 100%; color: blue;'><b>ICNPG2023 en Google Colaboratory-Instituto Balseiro</b></marquee>


# Preparativos para programar CUDA C/C++ en google colabs

Hola!, Bueno, aqui va un ejemplito de como correr codigo CUDA C/C++ en colabs
[[1]](https://https://www.wikihow.com/Run-CUDA-C-or-C%2B%2B-on-Jupyter-(Google-Colab)
[[2]](https://stackoverflow.com/questions/56854243/how-to-link-the-libraries-when-executing-cuda-program-on-google-colab).

No se olviden de Runtime-> Change Runtime Type -> GPU. Para que funque a cada linea la tienen que ejecutar con un Shift-Enter o Ctrl-Enter o el botoncito de play.

miremos que version de nvcc tenemos

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


A ver que placa nos toco...

In [ ]:
!nvidia-smi

Thu Apr 27 20:02:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

lindas GPUs!!. Ahora, para poder correr Cuda C/C++ instalamos un plugin

Veamos que cpu tenemos...

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

tiene dos procesadores, entonces no va a ser muy provechoso paralelizar en cpu...

Como vamos a programar en cuda C/C++ nos va a venir buen instalar un plugin, asi corremos sin compilar explicitamente, como si fuera python :-)

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter to /tmp/pip-req-build-mtuc9_od
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter /tmp/pip-req-build-mtuc9_od
  Resolved https://github.com/andreinechaev/nvcc4jupyter to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=07fc14b85531ccccd560004c939ced8e8e71587c75a71920608c576f18c3111f
  Stored in directory: /tmp/pip-ephem-wheel-cache-jmedw85k/wheels/65/6f/8e/80e37ad4f364ae56c9806a68974265420b0d623cc5434935d5
Successfully built NVCCPlugin


y luego cargarlo:

In [ ]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


Listo!, con eso ya podemos correr codigo CUDA C/C++ en el notebook Jupyter.

Para terminar, es conveniente montar nuestro google drive, asi podemos acceder a nuestros archivos, que pueden ser headers, etc. Nos va a pedir un permiso y un codigo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Suma de vectores

Solucion con bibliotecas. Para que ande hay que hacerlo en dos partes. Primero generar el codigo en una carpeta local del server:


In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
using namespace thrust::placeholders;

int main()
{
  thrust::device_vector<float> x(4), y(4);

  // Esto seria ilegal si x e y fueran simples punteros a memoria de device!  
  x[0] = 1;
  x[1] = 2;
  x[2] = 3;
  x[3] = 4;

  y[0] = 4;
  y[1] = 3;
  y[2] = 2;
  y[3] = 1;

  // algoritmos de thrust: toman rangos de uno o mas vectores y una operacion 

  thrust::transform(x.begin(), x.end(), y.begin(), y.begin(),
    _1 + _2
  );

  // y es ahora {5, 5, 5, 5} --> HANDS-ON: comprobar!
  for(int i=0;i<4;i++)
  std::cout << y[i] << std::endl; //o usar printf...

}


5
5
5
5



# SAXPY

In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <stdio.h>

using namespace thrust::placeholders;

int main()
{
  thrust::device_vector<float> x(4), y(4);
  x[0] = 2;
  x[1] = 4;
  x[2] = 6;
  x[3] = 8;

  y[0] = 4;
  y[1] = 3;
  y[2] = 2;
  y[3] = 1;
  float a=0.5;

  thrust::transform(x.begin(), x.end(), y.begin(), y.begin(),
    a*_1 + _2
  );
  // y es ahora {5, 5, 5, 5} --> HANDS-ON: comprobar!
  for(int i=0;i<4;i++)
  printf("%f\n", float(y[i]));
}



5.000000
5.000000
5.000000
5.000000



# Transformación arbitraria de dos arrays



In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <cmath>
#include <iostream>

struct mi_operacion
{
          __device__
          float operator()(float a, float b)
          {
                  return sqrt(a+b);
          }
};

int main()
{
  thrust::device_vector<float> x(4), y(4);
  x[0] = 1;
  x[1] = 2;
  x[2] = 3;
  x[3] = 4;

  y[0] = 4;
  y[1] = 3;
  y[2] = 2;
  y[3] = 1;

  thrust::transform(x.begin(), x.end(), y.begin(), y.begin(),mi_operacion());

  for(int i=0;i<4;i++)
  std::cout << y[i] << std::endl;
  
  std::cout << "sqrt(5)=" << sqrt(5.0) << std::endl;

}

2.23607
2.23607
2.23607
2.23607
sqrt(5)=2.23607



en esta caso no vamos a usar %%cu porque necesitamos compilar con algunas opciones (y no sabria como agregarlas en el plugin ese)

In [ ]:
%%cuda --name lambdas.cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <cmath>
#include <iostream>

struct mi_operacion
{
          __device__
          float operator()(float a, float b)
          {
                  return sqrt(a+b);
          }
};

int main()
{
    thrust::device_vector<float> x(4), y(4);
    x[0] = 1;
    x[1] = 2;
    x[2] = 3;
    x[3] = 4;

    y[0] = 4;
    y[1] = 3;
    y[2] = 2;
    y[3] = 1;

    thrust::transform(x.begin(), x.end(), y.begin(), y.begin(),
        [=] __device__ (float x1,float x2) 
        {
            return sqrt(x1 + x2);
        }
    );
  
    for(int i=0;i<4;i++)
    std::cout << y[i] << std::endl;

    std::cout << "sqrt(5)=" << sqrt(5.0) << std::endl;
}

'File written in /content/src/lambdas.cu'

In [ ]:
!nvcc -O2 --extended-lambda /content/src/lambdas.cu -o lambdas

In [ ]:
!/content/lambdas

2.23607
2.23607
2.23607
2.23607
sqrt(5)=2.23607


# Normalización de un vector

In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <iostream>

struct mi_operacion // normaliza por N
{
      public:
      float Norm;
      mi_operacion(float suma){Norm=suma;};
          __device__
          float operator()(float a)
          {
                  return a/Norm;
          }
};


int main()
{
  thrust::device_vector<float> x(4), y(4);
  x[0] = 2;
  x[1] = 4;
  x[2] = 3;
  x[3] = 1;

  float suma = thrust::reduce(x.begin(), x.end());
  thrust::transform(x.begin(), x.end(), x.begin(),mi_operacion(suma));
  // HANDS-ON: chequear que este normalizado...

  for(int i=0;i<4;i++)
  std::cout << x[i] << std::endl;
}

0.2
0.4
0.3
0.1



ahora probemos simplificar un poco la syntaxis, reemplazando el functor por placeholders

In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>

using namespace thrust::placeholders;

int main()
{
  thrust::device_vector<float> x(4);
  x[0] = 2;
  x[1] = 4;
  x[2] = 3;
  x[3] = 1;

  float suma = thrust::reduce(x.begin(), x.end());
  thrust::transform(x.begin(), x.end(), x.begin(),_1/suma);
  // HANDS-ON: chequear que este normalizado...

  for(int i=0;i<4;i++)
  std::cout << x[i] << std::endl;
}

0.2
0.4
0.3
0.1



Veamos ahora como se puede interoperar entre CUDA C/C++ y Thrust. Usamos thrust para manejar alocacion y copias de arrays en device pero el resto lo hacemos con cuda kernels y punteros crudos a memoria de device.

In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>

__global__ 
void kernel_normaliza(float *x, float suma, int dim)
{
    int id = threadIdx.x + (blockIdx.x * blockDim.x); 
    if (id < dim)
    {
         x[id] = x[id]/suma;
    }
}

int main()
{
  thrust::device_vector<float> x(4);

  x[0] = 2;
  x[1] = 4;
  x[2] = 3;
  x[3] = 1;

  float suma = thrust::reduce(x.begin(), x.end());

  float * x_ptr = thrust::raw_pointer_cast(&x[0]);

  kernel_normaliza<<<1,4>>>(x_ptr,suma,4); 
  // HANDS-ON: chequear que este normalizado...

  for(int i=0;i<4;i++)
  std::cout << x[i] << std::endl;
}

0.2
0.4
0.3
0.1



Ahora probemos al reves, la alocacion y copias las hacemos con CUDA, y al kernel lo reemplazamos por un algoritmo de thrust

In [ ]:
%%cu
#include <thrust/device_ptr.h>
#include <thrust/device_free.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>

using namespace thrust::placeholders;
int main(void)
{
  float *raw_ptr;
  cudaMalloc((void **)&raw_ptr, 4*sizeof(float));
  thrust::device_ptr<float> x(raw_ptr);

  x[0] = 2;
  x[1] = 4;
  x[2] = 3;
  x[3] = 1;

  float suma = thrust::reduce(x, x+4);
  thrust::transform(x,x+4,x,_1/suma);

  for(int i=0;i<4;i++) std::cout << x[i] << std::endl;

  thrust::device_free(x);
  return 0;
}

0.2
0.4
0.3
0.1



Normalicemos ahora no por la suma, pero por la norma euclidea

In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <cmath>

struct elevaalcuadrado
{
  __device__ float operator()(float x){
      return x*x;      
  }    
}; 

int main()
{
  thrust::device_vector<float> x(4);
  x[0] = 2;
  x[1] = 4;
  x[2] = 3;
  x[3] = 1;

  using namespace thrust::placeholders;

  // vamos a normalizar un vector por su norma

  thrust::device_vector<float> xx(4);
  thrust::transform(x.begin(), x.end(),xx.begin(),elevaalcuadrado());
  float norma = sqrt(thrust::reduce(xx.begin(),xx.end()));
  thrust::transform(x.begin(), x.end(), x.begin(),_1/norma);
 
  for(int i=0;i<4;i++)
  std::cout << x[i] << std::endl;

  // deberia dar esto:
  // for i in 2. 4. 3. 1.; do echo "$i/sqrt(2*2+4*4+3*3+1*1)" | bc -l; done
  /*    .36514837167011074230
        .73029674334022148461
        .54772255750516611345
        .18257418583505537115
  */

  return 0;
}


0.365148
0.730297
0.547723
0.182574



Lo anterior es ineficiente porque usa un vector auxiliar xx, y llama a tres kernels. Usando kernel-fusion se pueden usar solo dos kernels y evitar xx: 

In [ ]:
%%cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <cmath>

struct elevaalcuadrado
{
  __device__ float operator()(float x){
      return x*x;      
  }    
}; 

int main()
{
  thrust::device_vector<float> x(4);
  x[0] = 2;
  x[1] = 4;
  x[2] = 3;
  x[3] = 1;

  using namespace thrust::placeholders;

  // vamos a normalizar un vector por su norma

  float norma = sqrt( thrust::transform_reduce(
      x.begin(), x.end(),
      elevaalcuadrado(),
      0,thrust::plus<float>()) ); 
  
  thrust::transform(x.begin(), x.end(), x.begin(),_1/norma);
 
  for(int i=0;i<4;i++)
  std::cout << x[i] << std::endl;

  // deberia dar esto:
  // for i in 2. 4. 3. 1.; do echo "$i/sqrt(2*2+4*4+3*3+1*1)" | bc -l; done
  /*    .36514837167011074230
        .73029674334022148461
        .54772255750516611345
        .18257418583505537115
  */
  return 0;
}


0.365148
0.730297
0.547723
0.182574



# Portabilidad de Thrust! 

1 código, tres ejecutables

In [ ]:
%%cuda --name portable.cu

#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <cmath>
#include <iostream>

struct elevaalcuadrado
{
  __device__ float operator()(float x){
      return x*x;      
  }    
}; 

int main()
{
  thrust::device_vector<float> x(4);
  x[0] = 2;
  x[1] = 4;
  x[2] = 3;
  x[3] = 1;

  using namespace thrust::placeholders;

  // vamos a normalizar un vector por su norma

  float norma = sqrt( thrust::transform_reduce(
      x.begin(), x.end(),
      elevaalcuadrado(),
      0,thrust::plus<float>()) ); 
  
  thrust::transform(x.begin(), x.end(), x.begin(),_1/norma);
 
  for(int i=0;i<4;i++)
  std::cout << x[i] << std::endl;

  // deberia dar esto:
  // for i in 2. 4. 3. 1.; do echo "$i/sqrt(2*2+4*4+3*3+1*1)" | bc -l; done
  /*    .36514837167011074230
        .73029674334022148461
        .54772255750516611345
        .18257418583505537115
  */
  return 0;
}


'File written in /content/src/portable.cu'

Compilemos el codigo para que corra en GPU

In [ ]:
!nvcc -O2  /content/src/portable.cu -o corroenGPU

In [ ]:
!/content/corroenGPU

0.365148
0.730297
0.547723
0.182574


Ahora compilemos el *mismo* codigo pero que corra en CPU en forma serial. Para eso primero le cambiamos la extension.

In [ ]:
!cp /content/src/portable.cu /content/src/portable.cpp

y ahora lo compilamos con otro compilador, el usual para c++, el que seguro tienen en sus maquinas

In [ ]:
!g++ -O2 /content/src/portable.cpp -I/usr/local/cuda/include/ -DTHRUST_DEVICE_SYSTEM=THRUST_DEVICE_SYSTEM_CPP -o corroenCPUserial

In [ ]:
!/content/corroenCPUserial

0.365148
0.730297
0.547723
0.182574


y finalmente, recompilemos el *mismo* código, pero para que aproveche las CPU multicore, es decir que los algoritmos corran en paralelo en CPU!

In [ ]:
!g++ -O2 /content/src/portable.cpp -fopenmp -DTHRUST_DEVICE_SYSTEM=THRUST_DEVICE_SYSTEM_OMP -lgomp -I/usr/local/cuda/include/ -o corroenCPUparalelo

In [ ]:
!/content/corroenCPUparalelo

0.365148
0.730297
0.547723
0.182574


Queda como ejercicio correr algo que tenga mucho mas computo y comparar las performances de cada código... ¿qué esperan?

# Otros algoritmos y kernel fusion


El siguiente ejemplo muestra como generar numeros random en host, o como ordenarlos en device, todo usando thrust. El programa es entonces portable.

In [ ]:
#@title ordenar en device un vector random generado en host
%%cu

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <thrust/random.h>

int main() {
  // Generate 32M random numbers serially.
  thrust::default_random_engine rng(1337);
  thrust::uniform_int_distribution<int> dist;
  thrust::host_vector<int> h_vec(32 << 20);
  thrust::generate(h_vec.begin(), h_vec.end(), [&] { return dist(rng); });

  // Transfer data to the device.
  thrust::device_vector<int> d_vec = h_vec;

  // Sort data on the device.
  thrust::sort(d_vec.begin(), d_vec.end());

  // Transfer data back to host.
  thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());

  for(int i=0;i<10;i++)
  std::cout << h_vec[i] << " ";
  std::cout << std::endl;
}

23 88 106 108 110 196 363 576 693 768 



Cualculemos ahora la media 

$$\langle x \rangle = \sum_i x_i/N$$

y varianza 

$$\langle (x-\langle x \rangle)^2 \rangle = \sum_i (x_i-\langle x \rangle)^2/N$$

$$\langle (x-\langle x \rangle)^2 \rangle = \langle x^2 \rangle - \langle x \rangle^2$$

de un vector muy grande, de $N$ elementos. 


In [ ]:
#@title media y varianza de numeros random (simple)
%%cu 

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <thrust/random.h>
#include <chrono>


int main() {
  // Generate random data serially.
  thrust::default_random_engine rng(133);
  thrust::uniform_real_distribution<double> dist(-50.0, 50.0);
  thrust::host_vector<double> h_vec(32 << 20);
  thrust::generate(h_vec.begin(), h_vec.end(), [&] { return dist(rng); });

  // Transfer to device and compute the sum.
  thrust::device_vector<double> d_vec = h_vec;

  auto start = std::chrono::high_resolution_clock::now();

  double media = thrust::reduce(d_vec.begin(), d_vec.end(), double(0), thrust::plus<double>())/double(32<<20);

  using namespace thrust::placeholders;
  double var = thrust::transform_reduce(d_vec.begin(), d_vec.end(), (_1-media)*(_1-media), double(0), thrust::plus<double>())/double(32<<20);

  auto end = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end - start).count();


  std::cout << "el promedio da = " <<  media << std::endl;  
  std::cout << "la varianza da = " <<  var << std::endl;
  std::cout << "Time elapsed: " << duration << " microseconds" << std::endl;
}

el promedio da = -0.00730641
la varianza da = 833.438
Time elapsed: 4293 microseconds



In [ ]:
print(32<<20)
print(2<<20)

33554432
2097152


En el siguiente ejemplo intentaremos optimizar el programa fusionando las dos reducciones (media, varianza).

In [ ]:
#@title media y varianza de numeros random (kernel fussion)
%%writefile mediayvar.cu

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/reduce.h>
#include <thrust/tuple.h>
#include <thrust/functional.h>
#include <thrust/random.h>
#include <chrono>


int main() {
  // Generate random data serially.
  thrust::default_random_engine rng(133);
  thrust::uniform_real_distribution<double> dist(-50, 50.0);
  thrust::host_vector<double> h_vec(32 << 20);
  thrust::generate(h_vec.begin(), h_vec.end(), [&] { return dist(rng); });

  // Transfer to device and compute the sum.
  thrust::device_vector<double> d_vec = h_vec;

  auto start = std::chrono::high_resolution_clock::now();

  auto ambas = 
  thrust::transform_reduce
  (
      d_vec.begin(), d_vec.end(), 
      [=] __device__ (double x)
      {
          return thrust::make_tuple(x,x*x);          
      },
      thrust::make_tuple(0.,0.), 
      [=] __device__ (auto b, auto a){
          return 
          thrust::make_tuple(
            thrust::get<0>(a)+thrust::get<0>(b),
            thrust::get<1>(a)+thrust::get<1>(b)
          );          
      }     
  );

  auto end = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end - start).count();

  double media= thrust::get<0>(ambas)/double(32<<20);
  double var= thrust::get<1>(ambas)/double(32<<20)-media*media;

  std::cout << "el promedio da = " <<  media << std::endl;  
  std::cout << "la varianza da = " <<  var << std::endl;
  std::cout << "Time elapsed: " << duration << " microseconds" << std::endl;
}

Writing mediayvar.cu


In [ ]:
!nvcc --extended-lambda mediayvar.cu ; ./a.out

el promedio da = -0.00730641
la varianza da = 833.438
Time elapsed: 2095 microseconds
